In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QMainWindow, QDockWidget, QTextEdit, QPlainTextEdit
from PyQt5.QtCore import Qt, QTimer
import filepath
import pdb
from panda3d.core import (
    loadPrcFileData
)
from typing import Tuple
from qpanda3d import QShowBase, QPanda3DWidget, QControl, Synchronizer
from demos.physics_room import RoomScene
from panda3d_game.app import ControlShowBase, ContextShowBase, PhysicsShowBase
from vispyutil.canvas import SynchronizedCanvas
from vispyutil.showbase import CanvasBackgroundShowBase
import numpy as np
from vispy import app, scene, visuals,use
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne,
    newton, second
)
from util.physics import autocomplete_units, G_val, getG
from demos.ball_room import MassedBall,tmoon
from util.log import *
from style_config import *

In [3]:
class StarCanvas(SynchronizedCanvas):
    def __init__(self, n_stars, rho=10):
        SynchronizedCanvas.__init__(self)
        self.n_stars = n_stars 
        # randomly pick stars
        np.random.seed(0)  # todo: use torch random state
        cos_pos_theta = np.random.uniform(-1,1, n_stars)
        pos_theta = np.arccos(cos_pos_theta)
        pos_phi = np.random.uniform(0,2*np.pi, n_stars)
        sizes = np.random.uniform(0, 5, n_stars) 
        r = rho * np.sin(pos_theta)
        x = r * np.cos(pos_phi)
        y = r * np.sin(pos_phi)
        z = rho * np.cos(pos_theta)
        positions = np.vstack([x,y,z]).T
        scatter = scene.visuals.Markers()
        scatter.set_data(positions, edge_color=None, face_color='white', size=sizes, symbol='o')
        # Add scatter plot to the view
        self.view.add(scatter)


class StarScene(CanvasBackgroundShowBase):
    def __init__(self):
        # ControlShowBase.__init__(self)
        self.stars_canvas = StarCanvas(60000)
        CanvasBackgroundShowBase.__init__(self,self.stars_canvas)

        
class PlanetStarScene(StarScene,PhysicsShowBase):
    def __init__(self):
        StarScene.__init__(self)
        PhysicsShowBase.__init__(self)
        self.bullet_world.setGravity((0,0,0))
        self.unit = {
            "mass" : tonne,
            "length" : 100*meter,
            "time": 1 * second,
            # "force" : sp.Number(1e3) * newton
        }
        autocomplete_units(self.unit)
        self.planet1 = MassedBall(
            name="planet1",
            radius=1000*meter,
            mass=1e6*tonne,
            units=self.unit
        )
        self.planet1.reparentTo(self.render)
        self.planet1.set_texture(tmoon)
        self.planet1.setScale(10)
        self.planet1.setPos(0,10,0)

class PlanetStarSpace(PlanetStarScene, QControl):
    def __init__(self, isQt = True):
        import pdb
        QControl.__init__(self)
        PlanetStarScene.__init__(self)
        self.isQt = isQt
        if self.isQt:
            self.startQt()

import logging
class QLogHandler(logging.Handler):
    def __init__(self, text_edit: QPlainTextEdit):
        super().__init__()
        self.text_edit = text_edit

    def emit(self, record):
        msg = self.format(record)
        self.text_edit.appendPlainText(msg)

class ConsoleWidget(QPlainTextEdit):
    def __init__(self, title):
        QTextEdit.__init__(self,title)
        self.setObjectName("console")
    pass # set a console for it

class LoggerWidget(QPlainTextEdit):
    def __init__(self, title):
        QTextEdit.__init__(self,title)
        self.setObjectName("logger")
        self.setReadOnly(True)
        self.handlers = {}

    def add_level(self, level:int):
        if level not in self.handlers.keys():
            new_handler = QLogHandler(self)
            formatter = logging.Formatter(
                # FIXME
                '%(asctime)s - %(levelname)s - %(message)s'
            )
            new_handler.setFormatter(formatter)
            self.handlers[level] = new_handler
    
    pass



class BufferWidget:
    # each item has its type
    # for example, input history or output history
    # each has its str
    pass

class TextBuffer:
    # FIXME: use a queue instead
    # implement: write to a cache file
    # cached queue
    pass

class TextBrowserWidget:
    # browse a buffer
    pass


class MainWindow(QMainWindow):
    def __init__(self, FPS=60, stylesheet=styleSheet):
        super().__init__()
        self.FPS = FPS
        # Create three dock widgets
        self.dock_top_left = QDockWidget("Game Camera", self)
        self.dock_bottom_left = QDockWidget("Console", self)
        self.dock_right = QDockWidget("Logger", self)
        
        # Add the docks to the main window
        self.addDockWidget(Qt.LeftDockWidgetArea, self.dock_top_left)
        self.addDockWidget(Qt.RightDockWidgetArea, self.dock_right)
        # Split the left dock area vertically (top and bottom)
        self.splitDockWidget(self.dock_top_left, self.dock_bottom_left, Qt.Vertical)
        self.resizeDocks([self.dock_top_left, self.dock_bottom_left], [200, 200], Qt.Vertical)
        self.setWindowTitle("Three Dock Layout")
        self.resize(800, 600)
        self.panda3d = None
        self.synchronizer = Synchronizer(self.FPS)
        loadPrcFileData("", "window-type offscreen")
        
        self.dock_bottom_left.setWidget(ConsoleWidget("Bottom Left Content"))
        self.log_widget = LoggerWidget("Game Logs")
        self.dock_right.setWidget(self.log_widget)
        self.log_widget.add_level(GAME_LOG)
        Loggable.add_handlers_for_all(self.log_widget.handlers[GAME_LOG]) 
        self.startGame()
        self.panda3d.log("game start")
        self.setStyleSheet(styleSheet)
        if stylesheet is not None:
            self.dock_bottom_left.setStyleSheet(stylesheet)

    def startGame(self):
        self.panda3d = PlanetStarSpace()
        self.panda3d.log("create world")
        self.synchronizer.setShowBase(self.panda3d)
        self.pandaWidget = QPanda3DWidget(
            self.panda3d, 
            synchronizer=self.synchronizer
        )
        self.synchronizer.addWidget(self.pandaWidget)
        self.dock_top_left.setWidget(self.pandaWidget)
        self.synchronizer.start()
        self.panda_mouse_watcher = self.panda3d.mouseWatcherNode # FIXME: let a widget watch this
        self.pandaWidget.setFocus()
        
        # self.panda3d.cam_sensitivity = 0.01

    # todo: remove a widget

    

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1711322746054/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


SystemExit: 0

/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
from logging import Logger

In [5]:
help(Logger._log)cla

Help on function _log in module logging:

_log(self, level, msg, args, exc_info=None, extra=None, stack_info=False, stacklevel=1)
    Low-level logging routine which creates a LogRecord and then calls
    all the handlers of this logger to handle the record.



In [6]:
l = Logger("")

In [7]:
l.level

0

In [4]:
logging.NOTSET

0

In [5]:
help(Logger.parent)

AttributeError: type object 'Logger' has no attribute 'parent'